<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Импорт библиотек

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import re
#from scipy import stats as st

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import make_scorer, f1_score


In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
import catboost
import time

from tqdm import tqdm
# from keras.models import Model
# from keras.layers import LSTM, Dense, Masking, Input, Concatenate, Reshape
# from keras.metrics import AUC
# from keras.utils import pad_sequences, plot_model, set_random_seed
# from keras.preprocessing.sequence import pad_sequences
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix
from catboost import CatBoostClassifier, Pool, cv

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Загрузка файла с данными
total_scores = pd.read_csv('data_01_predict_progress/total_scores.csv')
tournament_scores = pd.read_csv('data_01_predict_progress/tournament_scores.csv')
tournaments = pd.read_csv('data_01_predict_progress/tournaments.csv')
units = pd.read_csv('data_01_predict_progress/units.csv')


In [7]:
units

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0
3,735,green,168.0
4,736,green,168.0
...,...,...,...
4591,10122,green,198.0
4592,5108,green,62.0
4593,5109,green,27.0
4594,5110,green,62.0


In [8]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [9]:
units['color'].unique()

array(['green', 'lime', nan], dtype=object)

In [10]:
units['color'].value_counts()

color
green    3800
lime      795
Name: count, dtype: int64

In [11]:
units['school_id'].value_counts()

school_id
62.0     348
198.0    345
111.0    255
38.0     241
192.0    124
        ... 
2.0        1
267.0      1
163.0      1
191.0      1
227.0      1
Name: count, Length: 239, dtype: int64

units
Юнит – спортивная единица. Мы работаем с одиночным катанием, в одном юните только один спортсмен. Если бы были пары/команды, то было б несколько спортсменов в юните.
- id: идентификатор юнита
- color: категория
- school_id: идентификатор школы

In [12]:
tournaments

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0
3,4,2090-10-18,2090-10-21,2.0
4,5,2090-10-21,2090-10-24,2.0
...,...,...,...,...
137,7114,2092-04-15,2092-04-20,1.0
138,7116,2091-10-24,2091-10-27,0.0
139,7117,2092-04-29,2092-05-03,0.0
140,7115,2092-04-23,2092-04-26,2.0


tournaments
Турнир состоит из нескольких категорий, оценки по категориям расписаны в total_scores
- id: идентификатор турнира
- date_start: дата начала
- date_end: дата завершения
- origin_id: место проведения


In [13]:
total_scores

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.20,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.40,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.00,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4
3,442030,524,4785,46.20,44.86,91.84,47.98,-1.0,3,4,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,3,292.42,3
4,442031,412,4785,44.20,42.59,89.73,48.14,-1.0,15,5,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,6,265.34,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,462405,34979,7116,21.67,26.02,46.62,21.10,-0.5,6,20,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,20,71.66,20
21297,462406,1995,7116,23.20,25.50,46.22,21.22,-0.5,7,21,Произвольная программа,q Прыжок приземлён в четверть e Явно неправиль...,21,71.27,21
21298,462407,34841,7116,21.83,25.00,45.46,20.96,-0.5,5,22,Произвольная программа,q Прыжок приземлён в четверть << Пониженный пр...,22,70.49,22
21299,462408,34980,7116,21.17,26.00,45.24,20.74,-1.5,2,23,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,24,68.22,24


Оценки за выступления по категориям и общие за турнир
- id: идентификатор выступления, джойнится с tournament_scores.total_score_id
- unit_id: идентификатор юнита, ключ к units.id
- tournament_id: идентификатор турнира, tournaments.id
- components_score: артистизм (мастерство, композиция, хореография)
- base_score: базовая оценка за элементы в выступлении (идеал)
- elements_score: реальная оценка всех выполненных элементов, base_score+goe
- decreasings_score: снижения оценок за ошибки
- total_score: components_score+elements_score+decreasings_score за выступление
- starting_place: жеребъёвка
- place: занятое место в категории category_name+segment_name
- segment_name: название сегмента
- info: комментарии и пояснения к оценке
- overall_place: итоговое место в турнире
- overall_total_score: итоговая оценка за весь турнир
- overall_place_str: комментарии, пояснения

In [14]:
display(tournament_scores)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19
3,4,1,CCoSp4,NaN,3.5,0.56,4.06
4,5,1,FCSp4,NaN,3.2,0.64,3.84
...,...,...,...,...,...,...,...
172153,352008,463407,CCoSp3,NaN,3.0,0.60,3.60
172154,352009,463407,2Lz,NaN,2.1,0.00,2.10
172155,352010,463407,2Lo,NaN,1.7,0.23,1.93
172156,352011,463407,StSq2,NaN,2.6,0.52,3.12


In [15]:
title=tournament_scores['title'].value_counts()

In [16]:
# выполняемые эдементы
title.head(100)

title
CCoSp4        8687
2A            8500
ChSq1         8301
CCoSp3        6064
StSq2         5097
              ... 
2A+1Eu+3S      292
2Lo+1A+SЕQ     286
3S<+2T         285
1F+1Lo         278
4T             277
Name: count, Length: 100, dtype: int64

In [17]:
tournament_scores['decrease'].unique()

array([nan, 'q', '<', '!', '<<', 'nS', 'F', '*', 'e', 'nU', 'nC', '<<*',
       '<*', 'B', 'nS*', '!F', 'F*', '!*', 'q*', 'e*', 'nF', 'nB', 'B.',
       'f*', 'В', '<F', 'qF', 'b', '!<', 'f', '!F*', 'FnU', 'FF'],
      dtype=object)

- '*' - элемент не засчитывается
- q - недокручена четверть
- < - недокручено 90-180 градусов
- << - недокручено больше 180 градусов
- e  - отталкивание не с того ребра
- ! - толчок не с точного ребра
- F - упал
- B - bonus 
- nU
- nC - не было либелы
- nS - не было волчка
- nF
- nB
- v -  можно заменить все ошибки во вращениях

In [18]:
tournament_scores['title'].nunique()

3425

tournament_scores Таблица с оценками поэлементно
- id: идентификатор оценки за конкретный элемент/комбинацию
- total_score_id: идентикатор выступления, ключ total_scores.id
- title: запись элемента или комбинации элементов с отметками об ошибках
- decrease: за что снижена оценка
- base_score: базовая оценка (идеал, цена данного элемента/комбинации, сложность)
- goe: Grade of Execute, качество исполнения, судейские надбавки/убавки
- avg_score: оценка за элемент/комбинацию (усредненная по судьям)

In [19]:
tournament_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB


В 'decrease'  много пропусков - это связано с тем, что замечаней по выполнению элементов не было

В "base_score" - присутствуют нули. Исправим это пересчитав значение столбца по формуле: "base_score"= "avg_score" - "goe"

In [20]:
tournament_scores['base_score'] = tournament_scores['avg_score'] - tournament_scores['goe']

In [21]:
tournament_scores.head(20)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.30,0.66,3.96
1,2,1,3F+3Lo,NaN,10.20,-0.11,10.09
2,3,1,3Lz,NaN,6.49,-1.30,5.19
3,4,1,CCoSp4,NaN,3.50,0.56,4.06
4,5,1,FCSp4,NaN,3.20,0.64,3.84
5,6,1,LSp4,NaN,2.70,0.49,3.19
6,7,1,StSq2,NaN,2.60,0.26,2.86
7,8,8,2A,NaN,3.30,0.53,3.83
8,9,8,3F,NaN,5.83,0.42,6.25
9,10,8,3Fq+3Loq,q,10.20,-1.91,8.29


In [22]:
#  figures = ["USp", "LSp", "CSp", "SSp", "FUSp", "FLSp", "FCSp", "FSSp", "CUSp", "CLSp", "CCSp",
#                          "CSSp", "CoSp", "CCoSp", "StSq", "ChSq", "2T", "2S", "2Lo", "2F", "2Lz", "2A", "3T", "3S",
#                          "3Lo", "3F", "3Lz", "3A", "4T", "4S", "4Lo", "4F", "4Lz", "4A"]

Объединим таблицы: units и total_scores

In [23]:
df = pd.merge(
    units, total_scores,
    left_on='id',
    right_on='unit_id'
)

In [24]:
df.sort_values('id_x', ascending=True)

,id_x,color,school_id,id_y,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
12334,1,green,198.0,7556,1,33,30.61,22.81,52.67,31.86,0.0,17,13,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во...,15,140.43,NaN
12337,1,green,198.0,10342,1,52,52.46,52.74,95.61,47.87,0.0,6,14,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,14,151.55,NaN
12336,1,green,198.0,441807,1,4790,52.39,55.65,111.05,55.40,0.0,28,11,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за пр...,10,170.40,10
12335,1,green,198.0,7576,1,33,48.03,43.79,87.76,44.97,0.0,6,15,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок x ...,15,140.43,NaN
12333,1,green,198.0,6094,1,21,59.26,55.67,118.09,62.42,0.0,5,4,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,4,174.38,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16201,35005,green,302.0,463517,35005,7117,12.54,14.19,26.95,12.76,0.0,1,7,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во...,0,0.00,NaN
16202,35005,green,302.0,463528,35005,7117,20.45,26.50,46.63,20.63,-0.5,5,7,Произвольная программа,< Недокрученный прыжок x Надбавка за прыжки во...,0,0.00,NaN
16430,35006,green,302.0,463619,35006,7117,30.43,38.71,67.63,29.92,-1.0,6,14,Произвольная программа,! Неясное ребро на толчке F/Lz x Надбавка за п...,0,0.00,NaN
16429,35006,green,302.0,463615,35006,7117,18.18,19.88,36.13,17.25,-1.0,14,21,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во...,0,0.00,NaN


In [25]:
df = df.drop(["id_x"], axis=1)

In [26]:
df = df.rename(columns = {'id_y':'total_score_id'})

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим таблицу tournaments

In [28]:
df = pd.merge(
    df, tournaments,
    left_on='tournament_id',
    right_on='id'
)

In [29]:
pd.set_option('display.max_colwidth', None)

In [30]:

pd.reset_option('display.max_columns')


In [31]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [32]:
df

,color,school_id,total_score_id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,id,date_start,date_end,origin_id
0,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,4785,2091-12-20,2091-12-24,2.0
1,green,244.0,442094,9474,4785,46.05,61.33,95.90,40.57,-6.0,10,18,Произвольная программа,* Недопустимый элемент < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,18,164.30,18,4785,2091-12-20,2091-12-24,2.0
2,green,203.0,5246,733,17,22.40,21.64,42.70,21.06,0.0,18,16,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),17,107.00,NaN,17,2090-12-09,2090-12-12,2.0
3,green,203.0,5271,733,17,32.98,36.93,64.30,28.37,0.0,8,18,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок e Явно неправильное ребро на толчке F/Lz x Надбавка за прыжки во второй половине программы (10%),17,107.00,NaN,17,2090-12-09,2090-12-12,2.0
4,green,235.0,442856,734,4793,21.95,22.76,41.76,21.00,-2.0,14,29,Короткая программа,< Недокрученный прыжок << Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,25,121.41,25,4793,2091-12-09,2091-12-11,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21296,green,198.0,457859,10121,7110,7.50,12.14,20.76,8.62,0.0,15,4,Произвольная программа,"Мастерство катания@@@1.50@@@4,00@@@4,00@@@3,75@@@3,75@@@4,00@@@3.92",4,20.76,4,7110,2092-04-09,2092-04-12,1.0
21297,green,198.0,455992,10121,6854,5.90,8.50,14.07,5.57,0.0,4,7,Произвольная программа,"Мастерство катания@@@1.50@@@2,75@@@2,75@@@3,00@@@2.83",7,14.07,7,6854,2091-12-06,2091-12-06,0.0
21298,green,198.0,457860,10122,7110,7.80,11.87,20.63,8.76,0.0,32,5,Произвольная программа,"Мастерство катания@@@1.50@@@4,25@@@3,75@@@4,00@@@4,00@@@4,25@@@4.08",5,20.63,5,7110,2092-04-09,2092-04-12,1.0
21299,green,198.0,455988,10122,6854,5.80,9.76,15.14,5.88,-0.5,5,3,Произвольная программа,"Мастерство катания@@@1.50@@@3,25@@@3,25@@@3,25@@@3.25",3,15.14,3,6854,2091-12-06,2091-12-06,0.0


удалим колонки дубликаты "id_x", "id"

In [33]:
df = df.drop(["id"], axis=1)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   color                21301 non-null  object 
 1   school_id            20384 non-null  float64
 2   total_score_id       21301 non-null  int64  
 3   unit_id              21301 non-null  int64  
 4   tournament_id        21301 non-null  int64  
 5   base_score           21301 non-null  float64
 6   components_score     21301 non-null  float64
 7   total_score          21301 non-null  float64
 8   elements_score       21301 non-null  float64
 9   decreasings_score    21301 non-null  float64
 10  starting_place       21301 non-null  int64  
 11  place                21301 non-null  int64  
 12  segment_name         21284 non-null  object 
 13  info                 20720 non-null  object 
 14  overall_place        21301 non-null  int64  
 15  overall_total_score  21284 non-null 

Добавим tournament_scores

In [35]:
df = pd.merge(
    df, tournament_scores,
    left_on='total_score_id',
    right_on='total_score_id'
)

In [36]:
df

,color,school_id,total_score_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score
0,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156595,3Lz+3T,NaN,10.10,1.18,11.28
1,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156596,2A,NaN,3.30,0.85,4.15
2,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156597,FCSp4,NaN,3.20,0.87,4.07
3,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156598,3F,NaN,5.83,-1.36,4.47
4,green,244.0,442068,9474,4785,31.73,33.87,68.40,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.30,18,2091-12-20,2091-12-24,2.0,156599,SSp4,NaN,2.50,0.61,3.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172153,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336176,1A,NaN,1.10,0.11,1.21
172154,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336177,CCoSp1,NaN,2.00,0.27,2.27
172155,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336178,ChSpl1,NaN,1.50,0.30,1.80
172156,green,198.0,461320,10122,6885,9.68,17.87,27.85,9.98,0.0,36,5,Произвольная программа.,< Недокрученный прыжок,5,27.85,5,2092-04-06,2092-04-08,0.0,336179,2S<,<,1.04,-0.21,0.83


In [37]:
df = df.drop(["total_score_id"], axis=1)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   color                172158 non-null  object 
 1   school_id            164913 non-null  float64
 2   unit_id              172158 non-null  int64  
 3   tournament_id        172158 non-null  int64  
 4   base_score_x         172158 non-null  float64
 5   components_score     172158 non-null  float64
 6   total_score          172158 non-null  float64
 7   elements_score       172158 non-null  float64
 8   decreasings_score    172158 non-null  float64
 9   starting_place       172158 non-null  int64  
 10  place                172158 non-null  int64  
 11  segment_name         172115 non-null  object 
 12  info                 168356 non-null  object 
 13  overall_place        172158 non-null  int64  
 14  overall_total_score  172115 non-null  float64
 15  overall_place_str

In [39]:
df['origin_id'].value_counts()

origin_id
2.0    78204
1.0    55240
0.0    38714
Name: count, dtype: int64

In [40]:
print('Количество турниров:', df['tournament_id'].nunique())

Количество турниров: 142


In [41]:
print('Количество записей по каждому турниру:')
df['tournament_id'].value_counts()

Количество записей по каждому турниру:


tournament_id
6859    4314
78      4223
7114    4045
6848    3660
77      3398
7098    3041
7117    2860
85      2853
7103    2838
79      2786
7110    2769
88      2749
7106    2635
6864    2634
6866    2433
2       2363
24      2255
39      2229
76      2222
68      2218
6699    2203
6885    2011
7092    1982
18      1909
3       1885
14      1873
4992    1863
176     1850
47      1767
40      1728
7111    1724
4790    1709
4793    1638
17      1617
45      1588
48      1550
7109    1547
73      1519
398     1484
6847    1454
84      1430
4789    1412
6845    1410
6160    1403
21      1399
5879    1377
51      1337
177     1296
7115    1281
4794    1266
52      1265
23      1257
4865    1247
33      1245
1       1221
69      1180
91      1160
6850    1159
6849    1155
49      1155
7112    1116
7104    1032
72      1019
222     1009
92      1008
44      1008
263     1001
6676     999
6844     987
43       975
29       960
50       957
2616     948
86       940
22       936
95       93

In [42]:
print('Количество записей по каждому участнику:')
df["unit_id"].value_counts()

Количество записей по каждому участнику:


unit_id
351      442
350      411
354      365
310      363
313      363
        ... 
9526       2
9533       2
9532       2
9521       2
10015      1
Name: count, Length: 3393, dtype: int64

Соберём новую таблицу для изучения данных

In [43]:
new_df = df[['unit_id', 'title', 'decrease', 'date_start', 'date_end' ,'tournament_id']]

In [44]:
new_df

,unit_id,title,decrease,date_start,date_end,tournament_id
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785
1,9474,2A,NaN,2091-12-20,2091-12-24,4785
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785
3,9474,3F,NaN,2091-12-20,2091-12-24,4785
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785
...,...,...,...,...,...,...
172153,10122,1A,NaN,2092-04-06,2092-04-08,6885
172154,10122,CCoSp1,NaN,2092-04-06,2092-04-08,6885
172155,10122,ChSpl1,NaN,2092-04-06,2092-04-08,6885
172156,10122,2S<,<,2092-04-06,2092-04-08,6885


In [45]:
display(new_df[new_df['unit_id']==9474])

,unit_id,title,decrease,date_start,date_end,tournament_id
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785
1,9474,2A,NaN,2091-12-20,2091-12-24,4785
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785
3,9474,3F,NaN,2091-12-20,2091-12-24,4785
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785
5,9474,StSq3,NaN,2091-12-20,2091-12-24,4785
6,9474,CCoSp4,NaN,2091-12-20,2091-12-24,4785
7,9474,3Lz,F,2091-12-20,2091-12-24,4785
8,9474,3F<,F,2091-12-20,2091-12-24,4785
9,9474,3Lo,NaN,2091-12-20,2091-12-24,4785


In [46]:
print('Количество турниров для каждого участника')
display(df.groupby('unit_id')['tournament_id'].nunique())

Количество турниров для каждого участника


unit_id
1        15
3        12
4         6
5         6
6        12
         ..
35003     1
35004     1
35005     1
35006     1
35024     1
Name: tournament_id, Length: 3393, dtype: int64

In [47]:
# Функция для извлечения элементов из строки
def extract_elements(text):
    # Паттерн для совпадения элементов, исключая строчную 'e', заглавные 'V' и 'B', и 'q' после 'A', 'T', 'F'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*\d*(?![TFA]q)*(?<![V])\b'
    pattern = r'\b\d*[A-Z][a-zA-Z]*\d*\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*\d*V\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*(?![VB])\d*(?![VTFA]q)\b'
   # pattern = r'\b\d*[A-Z][a-zA-Z]*(?<![VB])\d*(?![VTFA]q)*(?<![VB])\b'
    # Находим все совпадения
    matches = re.findall(pattern, text)
    # Удаляем 'e' внутри каждого совпадения и обрабатываем случаи с V или B в конце элемента
    final_matches = []
    for match in matches:
        # Убираем 'e'
        match = re.sub(r'e', '', match)
        # Удаляем V или B в конце элемента
        match = re.sub(r'[B]$', '', match)
        match = re.sub(r'[V]$', '', match)
        if re.search(r'[ATFo]q', match):
            # Убираем "q" после "A", "T", "F"
            final_matches.append(re.sub(r'q', '', match))
        else:
            final_matches.append(match)
    return final_matches


# Применение функции к каждому элементу столбца
new_df['list_title'] = new_df['title'].apply(extract_elements)

new_df['list_title'] = new_df['list_title'].apply(lambda x: [item for item in x if item not in ['COMBO', 'REP']])


In [48]:
print(new_df[new_df['title']=='1Eu'])

Empty DataFrame
Columns: [unit_id, title, decrease, date_start, date_end, tournament_id, list_title]
Index: []


In [49]:
new_df.query('unit_id == 1')

,unit_id,title,decrease,date_start,date_end,tournament_id,list_title
100447,1,2A,NaN,2091-12-25,2091-12-26,4844,[2A]
100448,1,3F!,!,2091-12-25,2091-12-26,4844,[3F]
100449,1,LSp4,NaN,2091-12-25,2091-12-26,4844,[LSp4]
100450,1,3Lz+2Lo,NaN,2091-12-25,2091-12-26,4844,"[3Lz, 2Lo]"
100451,1,FCSp4,NaN,2091-12-25,2091-12-26,4844,[FCSp4]
100452,1,StSq2,NaN,2091-12-25,2091-12-26,4844,[StSq2]
100453,1,CCoSp4,NaN,2091-12-25,2091-12-26,4844,[CCoSp4]
100454,1,3T,NaN,2091-12-25,2091-12-26,4844,[3T]
100455,1,3Lz+2T,NaN,2091-12-25,2091-12-26,4844,"[3Lz, 2T]"
100456,1,2A,NaN,2091-12-25,2091-12-26,4844,[2A]


In [50]:
new_df.head(20)

,unit_id,title,decrease,date_start,date_end,tournament_id,list_title
0,9474,3Lz+3T,NaN,2091-12-20,2091-12-24,4785,"[3Lz, 3T]"
1,9474,2A,NaN,2091-12-20,2091-12-24,4785,[2A]
2,9474,FCSp4,NaN,2091-12-20,2091-12-24,4785,[FCSp4]
3,9474,3F,NaN,2091-12-20,2091-12-24,4785,[3F]
4,9474,SSp4,NaN,2091-12-20,2091-12-24,4785,[SSp4]
5,9474,StSq3,NaN,2091-12-20,2091-12-24,4785,[StSq3]
6,9474,CCoSp4,NaN,2091-12-20,2091-12-24,4785,[CCoSp4]
7,9474,3Lz,F,2091-12-20,2091-12-24,4785,[3Lz]
8,9474,3F<,F,2091-12-20,2091-12-24,4785,[3F]
9,9474,3Lo,NaN,2091-12-20,2091-12-24,4785,[3Lo]


In [51]:
# Список новых фичей
new_features = ["USp", "LSp", "LSp1",'LSp2', 'LSp3', 'LSp4',
                "CSp", "CSp1", "CSp2", "CSp3", "CSp4",
                "SSp", "SSp1",  "SSp2", "SSp3", "SSp4",
                "FUSp",
                "FLSp1", 
                "FCSp", "FCSp1", "FCSp2", "FCSp3", "FCSp4",
                "FSSp", "FSSp1", "FSSp2", "FSSp3", "FSSp4", 
                "CUSp", 
                "CLSp",
                "CCSp", "CCSp1", "CCSp2", "CCSp3", "CCSp4", 
                "CSSp", "CSSp1", "CSSp2", "CSSp3", "CSSp4", 
                "CoSp", "CoSp1", "CoSp2", 
                "CCoSp", "CCoSp1", "CCoSp2", "CCoSp3", "CCoSp4", 
                "StSq", "StSq1", "StSq2", "StSq3", "StSq4", 
                "ChSq", "ChSq1",
                "T", "1T", "2T", "3T", "4T", 
                "S", "1S", "2S", "3S", "4S", 
                "Lo", "1Lo", "2Lo", "3Lo", "4Lo", 
                "F", "1F", "2F", "3F", "4F", 
                "Lz", "1Lz", "2Lz", "3Lz", "4Lz", 
                "A", "1A", "2A", "3A", "4A",
                "1Eu"]

# # Создание таблицы для новых фичей
# features = pd.DataFrame(columns=['unit_id'] + new_features)

# Создание списка для накопления данных
rows = []

# Заполнение таблицы
for unit_id in new_df['unit_id'].unique():
    # Получаем все записи для данного unit_id
    unit_records = new_df[new_df['unit_id'] == unit_id]
    decrease_value = unit_records['decrease'].iloc[0]  # Берем значение decrease из первой записи
    
    # Инициализация словаря для данного unit_id
    unit_dict = {'unit_id': unit_id}
    for feature in new_features:
        # Если 'decrease' равно NaN, устанавливаем значение 1 для всех фичей
        if pd.isna(decrease_value):
            unit_dict[feature] = 1 if any(feature in lst for lst in unit_records['list_title']) else 0
        # Если 'decrease' не равно NaN, устанавливаем значение 0 для всех фичей
        else:
            unit_dict[feature] = 0
    
    # Добавляем словарь в список
    rows.append(unit_dict)

# Преобразование списка словарей в DataFrame
target_df = pd.DataFrame(rows)







In [52]:
target_df.head(100)

,unit_id,USp,LSp,LSp1,LSp2,LSp3,LSp4,CSp,CSp1,CSp2,CSp3,CSp4,SSp,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp,FCSp1,FCSp2,FCSp3,FCSp4,FSSp,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp,CSSp1,CSSp2,CSSp3,CSSp4,CoSp,CoSp1,CoSp2,CCoSp,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq,StSq1,StSq2,StSq3,StSq4,ChSq,ChSq1,T,1T,2T,3T,4T,S,1S,2S,3S,4S,Lo,1Lo,2Lo,3Lo,4Lo,F,1F,2F,3F,4F,Lz,1Lz,2Lz,3Lz,4Lz,A,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1
1,733,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,737,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,0,0,0,1,0,0,1
6,738,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1
7,739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,740,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0
9,1007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
#объединим столбцы с одинаковыми по смыслу элементами: 
target_df['LSp1'] = target_df[['LSp', 'LSp1']].max(axis=1)
target_df['CSp1'] = target_df[['CSp', 'CSp1']].max(axis=1) 
target_df['SSp1'] = target_df[['SSp', 'SSp1']].max(axis=1)
target_df['FCSp1'] = target_df[['FCSp', 'FCSp1']].max(axis=1)
target_df['FSSp1'] = target_df[['FSSp', 'FSSp1']].max(axis=1)
target_df['CCSp1'] = target_df[['CCSp', 'CCSp1']].max(axis=1)
target_df['CSSp1'] = target_df[['CSSp', 'CSSp1']].max(axis=1)
target_df['CoSp1'] = target_df[['CoSp', 'CoSp1']].max(axis=1)
target_df['CCoSp1'] = target_df[['CCoSp', 'CCoSp1']].max(axis=1)
target_df['StSq1'] = target_df[['StSq', 'StSq1']].max(axis=1)
target_df['ChSq'] = target_df[['ChSq', 'ChSq1']].max(axis=1)
target_df['1T'] = target_df[['T', '1T']].max(axis=1)
target_df['1S'] = target_df[['S', '1S']].max(axis=1)
target_df['1Lo'] = target_df[['Lo', '1Lo']].max(axis=1)
target_df['1F'] = target_df[['F', '1F']].max(axis=1)
target_df['1Lz'] = target_df[['Lz', '1Lz']].max(axis=1) 
target_df['1A'] = target_df[['A', '1A']].max(axis=1)

In [54]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 87 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   unit_id  3393 non-null   int64
 1   USp      3393 non-null   int64
 2   LSp      3393 non-null   int64
 3   LSp1     3393 non-null   int64
 4   LSp2     3393 non-null   int64
 5   LSp3     3393 non-null   int64
 6   LSp4     3393 non-null   int64
 7   CSp      3393 non-null   int64
 8   CSp1     3393 non-null   int64
 9   CSp2     3393 non-null   int64
 10  CSp3     3393 non-null   int64
 11  CSp4     3393 non-null   int64
 12  SSp      3393 non-null   int64
 13  SSp1     3393 non-null   int64
 14  SSp2     3393 non-null   int64
 15  SSp3     3393 non-null   int64
 16  SSp4     3393 non-null   int64
 17  FUSp     3393 non-null   int64
 18  FLSp1    3393 non-null   int64
 19  FCSp     3393 non-null   int64
 20  FCSp1    3393 non-null   int64
 21  FCSp2    3393 non-null   int64
 22  FCSp3    3393 non-null  

In [55]:
target_df = target_df.drop(columns=['LSp', 'CSp', 'SSp', 'FCSp', 'FSSp', 'CCSp', 'CSSp', 'CoSp',
                                       'CCoSp', 'StSq', 'ChSq1', 'T', 'S', 'Lo', 'F', 'Lz', 'A' ])

In [56]:
display(target_df.head())

,unit_id,USp,LSp1,LSp2,LSp3,LSp4,CSp1,CSp2,CSp3,CSp4,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp1,FCSp2,FCSp3,FCSp4,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp1,CSSp2,CSSp3,CSSp4,CoSp1,CoSp2,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq1,StSq2,StSq3,StSq4,ChSq,1T,2T,3T,4T,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,1A,2A,3A,4A,1Eu
0,9474,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1
1,733,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
2,734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,735,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0
4,736,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3393 entries, 0 to 3392
Data columns (total 70 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   unit_id  3393 non-null   int64
 1   USp      3393 non-null   int64
 2   LSp1     3393 non-null   int64
 3   LSp2     3393 non-null   int64
 4   LSp3     3393 non-null   int64
 5   LSp4     3393 non-null   int64
 6   CSp1     3393 non-null   int64
 7   CSp2     3393 non-null   int64
 8   CSp3     3393 non-null   int64
 9   CSp4     3393 non-null   int64
 10  SSp1     3393 non-null   int64
 11  SSp2     3393 non-null   int64
 12  SSp3     3393 non-null   int64
 13  SSp4     3393 non-null   int64
 14  FUSp     3393 non-null   int64
 15  FLSp1    3393 non-null   int64
 16  FCSp1    3393 non-null   int64
 17  FCSp2    3393 non-null   int64
 18  FCSp3    3393 non-null   int64
 19  FCSp4    3393 non-null   int64
 20  FSSp1    3393 non-null   int64
 21  FSSp2    3393 non-null   int64
 22  FSSp3    3393 non-null  

In [58]:
df.head(20)

,color,school_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score
0,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156595,3Lz+3T,NaN,10.10,1.18,11.28
1,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156596,2A,NaN,3.30,0.85,4.15
2,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156597,FCSp4,NaN,3.20,0.87,4.07
3,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156598,3F,NaN,5.83,-1.36,4.47
4,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156599,SSp4,NaN,2.50,0.61,3.11
5,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156600,StSq3,NaN,3.30,0.90,4.20
6,green,244.0,9474,4785,31.73,33.87,68.4,35.53,-1.0,1,9,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),18,164.3,18,2091-12-20,2091-12-24,2.0,156601,CCoSp4,NaN,3.50,0.75,4.25
7,green,244.0,9474,4785,46.05,61.33,95.9,40.57,-6.0,10,18,Произвольная программа,* Недопустимый элемент < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,18,164.3,18,2091-12-20,2091-12-24,2.0,156857,3Lz,F,5.90,-2.95,2.95
8,green,244.0,9474,4785,46.05,61.33,95.9,40.57,-6.0,10,18,Произвольная программа,* Недопустимый элемент < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,18,164.3,18,2091-12-20,2091-12-24,2.0,156858,3F<,F,4.24,-2.12,2.12
9,green,244.0,9474,4785,46.05,61.33,95.9,40.57,-6.0,10,18,Произвольная программа,* Недопустимый элемент < Недокрученный прыжок REP Повторение прыжка не в каскаде x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,18,164.3,18,2091-12-20,2091-12-24,2.0,156859,3Lo,NaN,4.90,0.98,5.88


In [59]:
df.duplicated().sum()

0

In [60]:
df = df.sort_values('unit_id').reset_index()

In [61]:
df.head()

,index,color,school_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score
0,100514,green,198.0,1,7115,60.47,54.48,114.72,60.24,0.0,6,8,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%),7,174.24,7,2092-04-23,2092-04-26,2.0,350662,3F+2Lo,NaN,7.0,0.74,7.74
1,100542,green,198.0,1,6842,28.70,28.13,61.62,33.49,0.0,6,1,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),6,157.68,6,2091-10-19,2091-10-20,0.0,207939,2A,NaN,3.3,0.66,3.96
2,100543,green,198.0,1,6842,28.70,28.13,61.62,33.49,0.0,6,1,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),6,157.68,6,2091-10-19,2091-10-20,0.0,207940,3Lz,NaN,5.9,1.06,6.96
3,100544,green,198.0,1,6842,28.70,28.13,61.62,33.49,0.0,6,1,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),6,157.68,6,2091-10-19,2091-10-20,0.0,207941,LSp4,NaN,2.7,0.59,3.29
4,100545,green,198.0,1,6842,28.70,28.13,61.62,33.49,0.0,6,1,Короткая программа,x Надбавка за прыжки во второй половине программы (10%),6,157.68,6,2091-10-19,2091-10-20,0.0,207942,3F+2Lo,NaN,7.7,0.85,8.55


In [62]:
feature_df = df.groupby('unit_id')['tournament_id'].nunique().reset_index()

In [63]:
feature_df.rename(columns={'tournament_id': 'count_tournament'}, inplace=True)

In [64]:
feature_df.head()

,unit_id,count_tournament
0,1,15
1,3,12
2,4,6
3,5,6
4,6,12


In [65]:
# Преобразование столбца 'date_start' в формат datetime, обработка ошибок
df['date_start'] = pd.to_datetime(df['date_start'], errors='coerce')

In [66]:
first_tournament = df.groupby('unit_id')['date_start'].min().reset_index()

In [67]:
first_tournament.rename(columns={'date_start': 'first_tournament'}, inplace=True)

In [68]:
first_tournament

,unit_id,first_tournament
0,1,2090-04-19
1,3,2090-10-05
2,4,2090-10-26
3,5,2090-10-26
4,6,2090-10-26
...,...,...
3388,35003,2092-04-29
3389,35004,2092-04-29
3390,35005,2092-04-29
3391,35006,2092-04-29


In [69]:
feature_df = pd.merge(
    feature_df, first_tournament,
    left_on='unit_id',
    right_on='unit_id'
)

In [70]:
feature_df

,unit_id,count_tournament,first_tournament
0,1,15,2090-04-19
1,3,12,2090-10-05
2,4,6,2090-10-26
3,5,6,2090-10-26
4,6,12,2090-10-26
...,...,...,...
3388,35003,1,2092-04-29
3389,35004,1,2092-04-29
3390,35005,1,2092-04-29
3391,35006,1,2092-04-29


In [71]:
last_tournament = df.groupby('unit_id')['date_start'].max().reset_index()
last_tournament.rename(columns={'date_start': 'last_tournament'}, inplace=True)
feature_df = pd.merge(
    feature_df, last_tournament,
    left_on='unit_id',
    right_on='unit_id'
)

In [72]:
color = df.groupby('unit_id')['color'].max().reset_index()
#last_tournament.rename(columns={'date_start': 'last_tournament'}, inplace=True)
feature_df = pd.merge(
    feature_df, color,
    left_on='unit_id',
    right_on='unit_id'
)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 26 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   index                172158 non-null  int64         
 1   color                172158 non-null  object        
 2   school_id            164913 non-null  float64       
 3   unit_id              172158 non-null  int64         
 4   tournament_id        172158 non-null  int64         
 5   base_score_x         172158 non-null  float64       
 6   components_score     172158 non-null  float64       
 7   total_score          172158 non-null  float64       
 8   elements_score       172158 non-null  float64       
 9   decreasings_score    172158 non-null  float64       
 10  starting_place       172158 non-null  int64         
 11  place                172158 non-null  int64         
 12  segment_name         172115 non-null  object        
 13  info          

In [74]:
df = df.dropna(subset =['segment_name'])

In [75]:
df['segment_name'].unique()

array(['Произвольная программа', 'Короткая программа', 'Элементы',
       'Пpoизвoльнaя пpoгpaммa', 'Элeмeнты', 'Kopoткaя пpoгpaммa',
       'Кopoткaя пpoгpaммa', 'Прыжки', 'Произвольная программа.',
       'Пpoизвольная программа', 'Коpоткая программа',
       'Произвольнaя программа', 'Кoроткая программа',
       'Прoизвольная прoграмма', 'Пpoизвольнaя программа',
       'Интерпретация.', 'Интерпретация'], dtype=object)

In [76]:
df['segment_name'].value_counts()

segment_name
Произвольная программа     91935
Короткая программа         53525
Произвольная программа.    17091
Элементы                    3520
Пpoизвoльнaя пpoгpaммa      1581
Прыжки                       852
Кoроткая программа           700
Кopoткaя пpoгpaммa           659
Произвольнaя программа       473
Прoизвольная прoграмма       427
Kopoткaя пpoгpaммa           377
Элeмeнты                     341
Пpoизвольная программа       315
Коpоткая программа           126
Интерпретация.               111
Пpoизвольнaя программа        70
Интерпретация                 12
Name: count, dtype: int64

In [77]:
# переименуем значения - пиведём их к единому написанию
df['segment_name'] = df['segment_name'].replace({
    'Пpoизвoльнaя пpoгpaммa': 'Произвольная программа',
    'Произвольная программа.': 'Произвольная программа',
    'Пpoизвольнaя программа': 'Произвольная программа',
    'Пpoизвольная программа': 'Произвольная программа',
    'Прoизвольная прoграмма': 'Произвольная программа',
    'Пpoизвольнaя программа': 'Произвольная программа',
    'Произвольнaя программа': 'Произвольная программа',
    'Kopoткaя пpoгpaммa': 'Короткая программа',
    'Кopoткaя пpoгpaммa': 'Короткая программа',
    'Коpоткая программа': 'Короткая программа',
    'Кoроткая программа': 'Короткая программа',
    'Пpогpaммa': 'Программа',
    'Элeмeнты': 'Элементы',
    'Прыжки': 'Прыжки',
    'Интерпретация.': 'Интерпретация'
})

print(df['segment_name'].unique())

['Произвольная программа' 'Короткая программа' 'Элементы' 'Прыжки'
 'Интерпретация']


In [78]:
# Группировка по 'unit_id' и 'segment_name' и нахождение минимального значения 'place'
best_place = df.groupby(['unit_id', 'segment_name'])['place'].min().unstack().reset_index()

# Переименование столбцов
best_place.columns = ['unit_id', 'best_place_interpretation', 'best_place_short', 'best_place_free', 'best_place_jump', 'best_place_elements']
best_place.head()

,unit_id,best_place_interpretation,best_place_short,best_place_free,best_place_jump,best_place_elements
0,1,NaN,1.0,3.0,NaN,NaN
1,3,NaN,1.0,1.0,NaN,NaN
2,4,NaN,2.0,2.0,NaN,NaN
3,5,NaN,2.0,2.0,NaN,NaN
4,6,NaN,3.0,1.0,NaN,NaN


In [79]:
best_place = best_place.fillna(0)

In [80]:
feature_df = pd.merge(
    feature_df, best_place,
    left_on='unit_id',
    right_on='unit_id'
)

In [81]:
# Группировка по 'unit_id' и 'segment_name' и нахождение максимальное значения 'place'
worst_place = df.groupby(['unit_id', 'segment_name'])['place'].max().unstack().reset_index()

# Переименование столбцов
worst_place.columns = ['unit_id', 'worst_place_interpretation', 'worst_place_short', 'worst_place_free', 'worst_place_jump', 'worst_place_elements']

worst_place = worst_place.fillna(0)

feature_df = pd.merge(
    feature_df, worst_place,
    left_on='unit_id',
    right_on='unit_id'
)

In [82]:
feature_df.head()

,unit_id,count_tournament,first_tournament,last_tournament,color,best_place_interpretation,best_place_short,best_place_free,best_place_jump,best_place_elements,worst_place_interpretation,worst_place_short,worst_place_free,worst_place_jump,worst_place_elements
0,1,15,2090-04-19,2092-04-23,green,0.0,1.0,3.0,0.0,0.0,0.0,27.0,15.0,0.0,0.0
1,3,12,2090-10-05,2092-04-23,green,0.0,1.0,1.0,0.0,0.0,0.0,20.0,17.0,0.0,0.0
2,4,6,2090-10-26,2091-12-15,lime,0.0,2.0,2.0,0.0,0.0,0.0,27.0,26.0,0.0,0.0
3,5,6,2090-10-26,2092-04-14,green,0.0,2.0,2.0,0.0,0.0,0.0,17.0,21.0,0.0,0.0
4,6,12,2090-10-26,2092-04-23,green,0.0,3.0,1.0,0.0,0.0,0.0,13.0,15.0,0.0,0.0


In [83]:
# Группировка по 'unit_id' и 'segment_name' и нахождение минимальное значения 'total_score'
worst_total_score = df.groupby(['unit_id', 'segment_name'])['total_score'].min().unstack().reset_index()

# Переименование столбцов
worst_total_score.columns = ['unit_id', 'worst_total_score_interpretation', 'worst_total_score_short', 'worst_total_score_free', 'worst_total_score_jump', 'worst_total_score_elements']

worst_total_score = worst_total_score.fillna(0)

feature_df = pd.merge(
    feature_df, worst_total_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [84]:
# Группировка по 'unit_id' и 'segment_name' и нахождение максимальное значения 'total_score'
best_total_score = df.groupby(['unit_id', 'segment_name'])['total_score'].max().unstack().reset_index()

# Переименование столбцов
best_total_score.columns = ['unit_id', 'best_total_score_interpretation', 'best_total_score_short', 'best_total_score_free', 'best_total_score_jump', 'best_total_score_elements']

best_total_score = best_total_score.fillna(0)

feature_df = pd.merge(
    feature_df, best_total_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [85]:
max_goe = df.groupby('unit_id')['goe'].max().reset_index()
max_goe.rename(columns={'goe': 'max_goe'}, inplace=True)
feature_df = pd.merge(
    feature_df, max_goe,
    left_on='unit_id',
    right_on='unit_id'
)

In [86]:
min_goe = df.groupby('unit_id')['goe'].min().reset_index()
min_goe.rename(columns={'goe': 'min_goe'}, inplace=True)
feature_df = pd.merge(
    feature_df, min_goe,
    left_on='unit_id',
    right_on='unit_id'
)

In [87]:
min_avg_score = df.groupby('unit_id')['avg_score'].min().reset_index()
min_avg_score.rename(columns={'avg_score': 'min_avg_score'}, inplace=True)
feature_df = pd.merge(
    feature_df, min_avg_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [88]:
max_avg_score = df.groupby('unit_id')['avg_score'].max().reset_index()
max_avg_score.rename(columns={'avg_score': 'max_avg_score'}, inplace=True)
feature_df = pd.merge(
    feature_df, max_avg_score,
    left_on='unit_id',
    right_on='unit_id'
)

In [89]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           3382 non-null   int64         
 1   count_tournament                  3382 non-null   int64         
 2   first_tournament                  3382 non-null   datetime64[ns]
 3   last_tournament                   3382 non-null   datetime64[ns]
 4   color                             3382 non-null   object        
 5   best_place_interpretation         3382 non-null   float64       
 6   best_place_short                  3382 non-null   float64       
 7   best_place_free                   3382 non-null   float64       
 8   best_place_jump                   3382 non-null   float64       
 9   best_place_elements               3382 non-null   float64       
 10  worst_place_interpretation        3382 non-null 

In [90]:
# Проверка преобразования
feature_df['unit_id'].nunique()

3382

In [91]:
df[df['unit_id']==4]

,index,color,school_id,unit_id,tournament_id,base_score_x,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,date_start,date_end,origin_id,id,title,decrease,base_score_y,goe,avg_score
476,111088,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161897,2F,NaN,1.98,0.18,2.16
477,111089,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161898,CSSp4,NaN,3.00,0.60,3.60
478,111095,lime,246.0,4,4789,21.93,30.19,50.78,21.59,-1.0,5,7,Короткая программа,<< Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,7,155.97,7,2091-12-15,2091-12-17,2.0,165149,CCSp3,NaN,2.80,0.28,3.08
479,111091,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161900,3T,NaN,4.62,0.00,4.62
480,111092,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161901,FCCoSp3,NaN,3.00,0.30,3.30
481,111093,lime,246.0,4,4789,21.93,30.19,50.78,21.59,-1.0,5,7,Короткая программа,<< Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,7,155.97,7,2091-12-15,2091-12-17,2.0,165147,3T+3T<<,F,5.50,-2.10,3.40
482,111094,lime,246.0,4,4789,21.93,30.19,50.78,21.59,-1.0,5,7,Короткая программа,<< Пониженный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,7,155.97,7,2091-12-15,2091-12-17,2.0,165148,2Lz,NaN,2.10,0.07,2.17
483,111087,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161896,3T+1Eu+2S,NaN,6.60,0.56,7.16
484,111090,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161899,ChSq1,NaN,3.00,0.67,3.67
485,111086,lime,246.0,4,4794,44.40,64.64,109.69,46.05,-1.0,6,5,Произвольная программа,q Прыжок приземлён в четверть x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе,5,168.59,5,2091-12-01,2091-12-03,2.0,161895,CCoSp4,NaN,3.50,0.70,4.20


In [92]:
full_df = pd.merge(
    feature_df, target_df,
    left_on='unit_id',
    right_on='unit_id'
)

In [93]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 98 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   unit_id                           3382 non-null   int64         
 1   count_tournament                  3382 non-null   int64         
 2   first_tournament                  3382 non-null   datetime64[ns]
 3   last_tournament                   3382 non-null   datetime64[ns]
 4   color                             3382 non-null   object        
 5   best_place_interpretation         3382 non-null   float64       
 6   best_place_short                  3382 non-null   float64       
 7   best_place_free                   3382 non-null   float64       
 8   best_place_jump                   3382 non-null   float64       
 9   best_place_elements               3382 non-null   float64       
 10  worst_place_interpretation        3382 non-null 

In [94]:
# Изменение типа данных столбца 'timestamp' на string
full_df['first_tournament'] = full_df['first_tournament'].astype(str)
full_df['last_tournament'] = full_df['last_tournament'].astype(str)

Разделим датасет на тренировочный и тестовый и выделим таргет.

In [95]:
full_df = full_df.drop(columns=['first_tournament', 'last_tournament'], axis = 1)

In [96]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 96 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   unit_id                           3382 non-null   int64  
 1   count_tournament                  3382 non-null   int64  
 2   color                             3382 non-null   object 
 3   best_place_interpretation         3382 non-null   float64
 4   best_place_short                  3382 non-null   float64
 5   best_place_free                   3382 non-null   float64
 6   best_place_jump                   3382 non-null   float64
 7   best_place_elements               3382 non-null   float64
 8   worst_place_interpretation        3382 non-null   float64
 9   worst_place_short                 3382 non-null   float64
 10  worst_place_free                  3382 non-null   float64
 11  worst_place_jump                  3382 non-null   float64
 12  worst_

In [97]:
# закодируем категориальные признаки столбец 'color'
# Кодирование категориальных признаков с использованием LabelEncoder
label_encoders = {}
for column in ['color']:
    le = LabelEncoder()
    full_df[column] = le.fit_transform(full_df[column])
    label_encoders[column] = le
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3382 entries, 0 to 3381
Data columns (total 96 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   unit_id                           3382 non-null   int64  
 1   count_tournament                  3382 non-null   int64  
 2   color                             3382 non-null   int64  
 3   best_place_interpretation         3382 non-null   float64
 4   best_place_short                  3382 non-null   float64
 5   best_place_free                   3382 non-null   float64
 6   best_place_jump                   3382 non-null   float64
 7   best_place_elements               3382 non-null   float64
 8   worst_place_interpretation        3382 non-null   float64
 9   worst_place_short                 3382 non-null   float64
 10  worst_place_free                  3382 non-null   float64
 11  worst_place_jump                  3382 non-null   float64
 12  worst_

In [98]:
#выделим таргет 
target = full_df[["USp", "LSp1",'LSp2', 'LSp3', 'LSp4',
                "CSp1", "CSp2", "CSp3", "CSp4",
                "SSp1",  "SSp2", "SSp3", "SSp4",
                "FUSp",
                "FLSp1", 
                "FCSp1", "FCSp2", "FCSp3", "FCSp4",
                "FSSp1", "FSSp2", "FSSp3", "FSSp4", 
                "CUSp", 
                "CLSp",
                "CCSp1", "CCSp2", "CCSp3", "CCSp4", 
                "CSSp1", "CSSp2", "CSSp3", "CSSp4", 
                "CoSp1", "CoSp2", 
                "CCoSp1", "CCoSp2", "CCoSp3", "CCoSp4", 
                "StSq1", "StSq2", "StSq3", "StSq4", 
                "ChSq",
                "1T", "2T", "3T", "4T", 
                "1S", "2S", "3S", "4S", 
                "1Lo", "2Lo", "3Lo", "4Lo", 
                "1F", "2F", "3F", "4F", 
                "1Lz", "2Lz", "3Lz", "4Lz", 
                "1A", "2A", "3A", "4A",
                "1Eu"]]
features = full_df.drop(columns=["USp", "LSp1",'LSp2', 'LSp3', 'LSp4',
                "CSp1", "CSp2", "CSp3", "CSp4",
                "SSp1",  "SSp2", "SSp3", "SSp4",
                "FUSp",
                "FLSp1", 
                "FCSp1", "FCSp2", "FCSp3", "FCSp4",
                "FSSp1", "FSSp2", "FSSp3", "FSSp4", 
                "CUSp", 
                "CLSp",
                "CCSp1", "CCSp2", "CCSp3", "CCSp4", 
                "CSSp1", "CSSp2", "CSSp3", "CSSp4", 
                "CoSp1", "CoSp2", 
                "CCoSp1", "CCoSp2", "CCoSp3", "CCoSp4", 
                "StSq1", "StSq2", "StSq3", "StSq4", 
                "ChSq",
                "1T", "2T", "3T", "4T", 
                "1S", "2S", "3S", "4S", 
                "1Lo", "2Lo", "3Lo", "4Lo", 
                "1F", "2F", "3F", "4F", 
                "1Lz", "2Lz", "3Lz", "4Lz", 
                "1A", "2A", "3A", "4A",
                "1Eu"], axis = 1)

In [99]:
target.describe()

,USp,LSp1,LSp2,LSp3,LSp4,CSp1,CSp2,CSp3,CSp4,SSp1,SSp2,SSp3,SSp4,FUSp,FLSp1,FCSp1,FCSp2,FCSp3,FCSp4,FSSp1,FSSp2,FSSp3,FSSp4,CUSp,CLSp,CCSp1,CCSp2,CCSp3,CCSp4,CSSp1,CSSp2,CSSp3,CSSp4,CoSp1,CoSp2,CCoSp1,CCoSp2,CCoSp3,CCoSp4,StSq1,StSq2,StSq3,StSq4,ChSq,1T,2T,3T,4T,1S,2S,3S,4S,1Lo,2Lo,3Lo,4Lo,1F,2F,3F,4F,1Lz,2Lz,3Lz,4Lz,1A,2A,3A,4A,1Eu
count,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.0,3382.0,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.0,3382.0,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.000000,3382.00000,3382.000000,3382.000000,3382.000000,3382.000000,3382.0,3382.000000
mean,0.002070,0.057067,0.092549,0.169426,0.160556,0.146363,0.005027,0.002957,0.002070,0.161443,0.032229,0.034891,0.005322,0.0,0.0,0.039030,0.082496,0.126552,0.120639,0.098462,0.161443,0.198699,0.142224,0.0,0.0,0.012714,0.013601,0.029864,0.028977,0.135719,0.062685,0.067120,0.054701,0.012419,0.000296,0.151390,0.169722,0.311059,0.259610,0.301597,0.245713,0.115316,0.016854,0.432584,0.038143,0.373448,0.195151,0.017445,0.046127,0.318746,0.248965,0.013897,0.237433,0.348906,0.196925,0.001478,0.135127,0.320520,0.166765,0.002070,0.221762,0.299527,0.17120,0.005914,0.364281,0.318155,0.029568,0.0,0.187463
std,0.045454,0.232005,0.289842,0.375184,0.367175,0.353522,0.070731,0.054304,0.045454,0.367993,0.176635,0.183530,0.072770,0.0,0.0,0.193695,0.275159,0.332520,0.325755,0.297983,0.367993,0.399080,0.349331,0.0,0.0,0.112055,0.115846,0.170237,0.167766,0.342540,0.242431,0.250267,0.227430,0.110761,0.017195,0.358481,0.375444,0.462995,0.438485,0.459019,0.430573,0.319451,0.128743,0.495508,0.191570,0.483791,0.396375,0.130943,0.209790,0.466059,0.432477,0.117081,0.425573,0.476695,0.397734,0.038427,0.341910,0.466746,0.372821,0.045454,0.415493,0.458119,0.37674,0.076684,0.481299,0.465829,0.169418,0.0,0.390340
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [100]:
target.sum()

USp          7
LSp1       193
LSp2       313
LSp3       573
LSp4       543
CSp1       495
CSp2        17
CSp3        10
CSp4         7
SSp1       546
SSp2       109
SSp3       118
SSp4        18
FUSp         0
FLSp1        0
FCSp1      132
FCSp2      279
FCSp3      428
FCSp4      408
FSSp1      333
FSSp2      546
FSSp3      672
FSSp4      481
CUSp         0
CLSp         0
CCSp1       43
CCSp2       46
CCSp3      101
CCSp4       98
CSSp1      459
CSSp2      212
CSSp3      227
CSSp4      185
CoSp1       42
CoSp2        1
CCoSp1     512
CCoSp2     574
CCoSp3    1052
CCoSp4     878
StSq1     1020
StSq2      831
StSq3      390
StSq4       57
ChSq      1463
1T         129
2T        1263
3T         660
4T          59
1S         156
2S        1078
3S         842
4S          47
1Lo        803
2Lo       1180
3Lo        666
4Lo          5
1F         457
2F        1084
3F         564
4F           7
1Lz        750
2Lz       1013
3Lz        579
4Lz         20
1A        1232
2A        1076
3A        

Удалим из таргета элементы, которые ни разу не встречались: 4A, CLSp, CUSp, FLSp1, FUSp и CoSp2 который встречался только один раз

In [101]:
target = target.drop(columns=[
                "FUSp",
                "FLSp1", 
                "CUSp", 
                "CLSp", "CoSp2",
                "4A"], axis = 1)

In [103]:
# Преобразование меток в формат мультилейбл
mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(target.values)

In [104]:
# разделим на тестовую и тренировочную 
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            Y, test_size=0.25, stratify=Y,
                                                                            random_state=123456)

In [105]:
# Создание модели
base_clf = LogisticRegression(max_iter=1000)
clf = MultiOutputClassifier(base_clf, n_jobs=-1)

# Определение метрики для кросс-валидации (используем средний F1-score для каждой метки)
def f1_micro_average(y_true, y_pred):
    return f1_score(y_true, y_pred, average='micro')

scorer = make_scorer(f1_micro_average)

# Кросс-валидация
scores = cross_val_score(clf, features_train, target_train, cv=5, scoring=scorer)

print(f'F1-score (micro average) на кросс-валидации: {scores}')
print(f'Средний F1-score (micro average) на кросс-валидации: {scores.mean()}')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parall

In [127]:
# Создание модели
base_clf = LogisticRegression(max_iter=1000)
clf = MultiOutputClassifier(base_clf, n_jobs=-1)

# Определение метрики для кросс-валидации (используем средний F1-score для каждой метки)
scorer = make_scorer(f1_score, average='micro')

# Кросс-валидация
scores = cross_val_score(clf, features_train, target_train, cv=5, scoring=scorer)

print(f'F1-score (micro average) на кросс-валидации: {scores}')
print(f'Средний F1-score (micro average) на кросс-валидации: {scores.mean()}')

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parall

In [107]:
# Создание модели CatBoost
base_clf = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0)
clf = MultiOutputClassifier(base_clf, n_jobs=-1)

# Определение метрики для кросс-валидации (используем средний F1-score для каждой метки)
scorer = make_scorer(f1_score, average='micro')

# Кросс-валидация
scores = cross_val_score(clf, features_train, target_train, cv=5, scoring=scorer)

print(f'F1-score (micro average) на кросс-валидации: {scores}')
print(f'Средний F1-score (micro average) на кросс-валидации: {scores.mean()}')


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 463, in _process_worker
    r = call_item()
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 291, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py", line 598, in __call__
    return [func(*args, **kwargs)
  File "/Users/tatianabeliaeva/opt/anaconda3/lib/python3.9/site-packages/joblib/parall